In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import seaborn as sns
import random

random.seed(10)

In [12]:
input_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/data"

In [13]:

df = pd.read_csv(os.path.join(input_dir,r"welfare_data_test.csv"))

In [14]:
df[df["year"]<=2016]["shareid"].nunique()

513

In [17]:
df["monthly_review_count"].sum()

497343.0

In [18]:

alpha = -0.01510385724693177
sigma = 0.5396
rating_param = 0.063

occ_upper = 95

In [19]:
df["mc_t"] = df["ADR"]+1/(alpha*(1/(1-sigma)-\
                                (sigma*df["group_share"]/(1-sigma))-\
                                df["market_share"]))

In [20]:
df["occ_greater_95"] = df["Occ"]>occ_upper

In [23]:
# calculate Occ=95% quantity
df["q_95"] = df["Supply"]*occ_upper/100

In [24]:
test = df[df["occ_greater_95"]==1]
test[test["Demand"]<test["q_95"]]

,index,post_month,shareid,year_month,Supply,Demand,Revenue,Occ,ADR,RevPAR,...,ln_group_share,quality,Rating,ReviewCount_Reviewed,Rating_ReviewCount_Reviewed,market_year_month,ln_ADR,mc_t,occ_greater_95,q_95


In [25]:
df["excess_demand"] = df["Demand"]-df["q_95"]

In [26]:
df["excess_demand_interaction"] = df["excess_demand"]*df["occ_greater_95"]

In [40]:
from statsmodels.api import add_constant
from linearmodels import IV2SLS
OH = OneHotEncoder(handle_unknown='ignore')

dep=df["mc_t"]

fe_var= ["shareid","market_year_month"]

x_fe = OH.fit_transform(df[fe_var]).toarray()
fe_var_cols = OH.fit(df[fe_var]).get_feature_names_out().tolist()
fe_var_df_1 = pd.DataFrame(x_fe, columns=fe_var_cols)
fe_var_df_1 = fe_var_df_1.drop(["shareid_75909",\
                                "market_year_month_2016-01-01Miami/Hialeah, FL",\
                               "market_year_month_2016-01-01Houston, TX",\
                               "market_year_month_2016-01-01Chicago, IL"], axis=1)

endog=df[["excess_demand_interaction"]]
endog = add_constant(endog)

exog = pd.concat([endog[["const"]],fe_var_df_1], axis=1).astype(float)

#df["ln_group_share"] = np.log(df["group_share"])

#from sklearn.preprocessing import StandardScaler
x =  df[[
                "hotel_count_in_group_chain",
               "hotel_count_in_group_franchised",\
               "hotel_count_in_group_independent",\
                "Group Avg Age",
               '1_size','2_size','3_size','4_size']]
#x = add_constant(x)
instr = x.to_numpy()
#inst_var = statsmodels.tools.tools.add_constant(inst_var)
#scaler = StandardScaler().fit(inst_var)
#inst_var = scaler.transform(inst_var)

model = IV2SLS(dep, exog, endog[["excess_demand_interaction"]], instr).fit(cov_type="unadjusted") 
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                   mc_t   R-squared:                      0.7803
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7768
No. Observations:               73118   F-statistic:                 3.114e+05
Date:                Mon, Nov 28 2022   P-value (F-stat)                0.0000
Time:                        22:59:28   Distribution:               chi2(1166)
Cov. Estimator:            unadjusted                                         
                                                                              
                                               Parameter Estimates                                               
=================================================================================================================
                                               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------------------------------
const                                             66.409     11.286     5.8843     0.0000      44.289      88.529
shareid_80022                                    -93.594     13.530    -6.9176     0.0000     -120.11     -67.076
shareid_80596                                     12.221     12.220     1.0001     0.3173     -11.730      36.171
shareid_80713                                    -21.444     13.133    -1.6328     0.1025     -47.185      4.2961
shareid_81037                                     5.9752     12.517     0.4774     0.6331     -18.557      30.508
shareid_81154                                     1.2369     12.141     0.1019     0.9189     -22.559      25.032
shareid_81425                                     10.062     10.712     0.9393     0.3476     -10.933      31.057
shareid_81685                                     73.819     10.966     6.7316     0.0000      52.326      95.312
shareid_81769                                     62.005     12.175     5.0930     0.0000      38.143      85.867
shareid_82026                                    -3.1954     12.284    -0.2601     0.7948     -27.271      20.880
shareid_82381                                     10.015     14.344     0.6982     0.4850     -18.098      38.128
shareid_82442                                     126.37     10.670     11.843     0.0000      105.45      147.28
shareid_82557                                     101.29     14.543     6.9647     0.0000      72.787      129.80
shareid_82658                                     21.564     13.838     1.5584     0.1191     -5.5573      48.685
shareid_83062                                     134.66     13.773     9.7768     0.0000      107.66      161.65
shareid_83265                                     21.540     12.145     1.7735     0.0761     -2.2644      45.344
shareid_83582                                     37.175     12.361     3.0075     0.0026      12.948      61.401
shareid_83784                                     100.54     12.526     8.0267     0.0000      75.990      125.09
shareid_84076                                     17.302     12.328     1.4035     0.1605     -6.8600      41.464
shareid_84172                                    -64.518     12.215    -5.2818     0.0000     -88.458     -40.577
shareid_84259                                     323.30     11.070     29.205     0.0000      301.61      345.00
shareid_84581                                     90.362     13.557     6.6655     0.0000      63.792      116.93
shareid_84600                                     14.941     12.746     1.1722     0.2411     -10.041      39.924
shareid_84681                                    -40.296     12.288    -3.2794     0.0010     -64.380     -16.213
shareid_84754                              

In [38]:
fe_var_df_1

,const,shareid_75909,shareid_80022,shareid_80596,shareid_80713,shareid_81037,shareid_81154,shareid_81425,shareid_81685,shareid_81769,...,"market_year_month_2017-09-01Miami/Hialeah, FL","market_year_month_2017-10-01Chicago, IL","market_year_month_2017-10-01Houston, TX","market_year_month_2017-10-01Miami/Hialeah, FL","market_year_month_2017-11-01Chicago, IL","market_year_month_2017-11-01Houston, TX","market_year_month_2017-11-01Miami/Hialeah, FL","market_year_month_2017-12-01Chicago, IL","market_year_month_2017-12-01Houston, TX","market_year_month_2017-12-01Miami/Hialeah, FL"
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73113,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73114,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73115,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73116,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
